In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

In [2]:
df = pd.read_csv('data/weatherAUS.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## Dealing with Missing Values

In [ ]:
#drop date, evaporation, sunshine(outliers),

In [3]:
df["Date"] = pd.to_datetime(df["Date"])   #Converting into Date
df["Year"] = df["Date"].dt.year           #abstracting year in different column 
df["Month"] = df["Date"].dt.month         #abstracting month in diffrent column  
df["Day"] = df["Date"].dt.day  

In [4]:
df.isna().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
Year                 0
Month                0
Day                  0
dtype: int64

In [ ]:
df.MinTemp.value_counts()
# since the mintemp has few missing values we can just take the average 
# and fill 

In [ ]:
df.MaxTemp.value_counts()
# Same with MinTemp

In [ ]:
df.Rainfall.value_counts()
#fill missing values with 0's (which means no rain that day)

In [ ]:
df.RainToday.value_counts()
# For both rain today and rain tommorow we can drop missing values
# they account for less than 3 percent

In [ ]:
df.Cloud3pm.value_counts()
# 9 is a typo, we can swap that to 8
# 0 means clear skies and we can't really input that data without skewing

In [ ]:
df.Cloud9am.value_counts()
#swap 9 to 8, probably use median or mean to input values, edit: nah that shit
#aint gonna work chief

# drop columns
Evaporation

In [ ]:
#X = df.loc['Cloud9am'].values
  
#from sklearn.impute import SimpleImputer
#imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
#imputer = imputer.fit(X)
  
#X = imputer.transform(X)
#print(X)

In [ ]:
X=df.drop(columns=['class'])
y=df['class']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)